In [1]:
import verde as vd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pyproj
from shapely import geometry


%matplotlib widget

gdb = '/home/ggrl/geodatabase/'

In [3]:
m_rparaim = pd.read_csv(gdb+'geof/recortado/m_rparaim')

In [2]:
g_rparaim = pd.read_csv(gdb+'geof/recortado/g_rparaim')

In [ ]:
g_rparaim.sort_values('LATITUDE')

In [ ]:
m_rparaim.sort_values('MAGIGRF')

In [3]:
coordinates = (g_rparaim.X.values, g_rparaim.Y.values)
coordinates

(array([500020.25, 500103.83, 500187.24, ..., 554618.94, 554618.52,
        554617.97]),
 array([8786155.43, 8786158.77, 8786161.64, ..., 8839028.43, 8839107.8 ,
        8839187.24]))

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c    = g_rparaim.CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

In [ ]:
g_rparaim['geometry'] = [geometry.Point(x,y) for x, y in zip(g_rparaim['LONGITUDE'], g_rparaim['LATITUDE'])]

In [ ]:
# Block Reduction

In [ ]:
'''
reducer_median = vd.BlockReduce(np.median, spacing= 500)
b_coords, b_CTCOR = reducer_median.filter(coordinates, g_rparaim.CTCOR)

plt.figure(figsize=(8,8))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()
'''

In [9]:
reducer_mean = vd.BlockReduce(np.mean, spacing= 1000)
b_coords, b_CTCOR = reducer_mean.filter(coordinates, g_rparaim.CTCOR)

plt.figure(figsize=(8,8))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
spline = vd.Spline()

In [11]:
spline.fit(b_coords, b_CTCOR)

Spline()

In [12]:
predicted = spline.predict(coordinates)

In [13]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='magma',
            s=1)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
residuals = g_rparaim.CTCOR - predicted

In [15]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 200,pixel_register=True)

In [91]:
region

(500005.3, 554693.86, 8783985.23, 8839303.77)

In [50]:
grid_CTCOR = spline.predict(grid_coords)

In [51]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_CTCOR,
            cmap='magma',
            s=2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
grid = spline.grid(spacing=200, data_names=['CTCOR'])
grid

<xarray.Dataset>
Dimensions:   (easting: 271, northing: 273)
Coordinates:
  * easting   (easting) float64 5.003e+05 5.005e+05 ... 5.542e+05 5.544e+05
  * northing  (northing) float64 8.784e+06 8.785e+06 ... 8.839e+06 8.839e+06
Data variables:
    CTCOR     (northing, easting) float64 1.618e+03 1.614e+03 ... 1.131e+03
Attributes:
    metadata:  Generated by Spline()

In [53]:
grid.CTCOR.plot(figsize=(8,8),
               cmap='magma')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500225.1398596865, 554503.805909544, 8784373.113641337, 8838919.583069772)

In [54]:
grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid)
grid

grid.CTCOR.plot(figsize=(8,8),cmap='magma')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500225.1398596865, 554503.805909544, 8784373.113641337, 8838919.583069772)

In [89]:
chain_CTCOR = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.median, spacing=1000),
    ('spline', vd.Spline()),
])

In [90]:
chain_CTCOR.fit(coordinates, g_rparaim.CTCOR)

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(center_coordinates=True,
                          reduction=<function median at 0x7f0919b19d30>,
                          spacing=1000)),
             ('spline', Spline())])

In [6]:
grid_CTCOR = chain_CTCOR.grid(spacing=200, data_names=['CTCOR'])
CTCOR_grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid_CTCOR)

CTCOR_grid.CTCOR.plot(figsize=(8,8), cmap='magma')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(499905.13780219783, 554794.0221978022, 8783885.377039712, 8839403.622960288)

Model Validation

In [20]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR,
                                 test_size = 0.1)

In [33]:
train

((array([513636.67, 505626.26, 504600.14, ..., 505129.17, 551611.51,
         501107.91]),
  array([8835956.24, 8822981.98, 8803658.98, ..., 8788523.72, 8830277.03,
         8827834.64])),
 (array([ 386.34, 1465.85,  682.07, ..., 1499.95, 1127.24,  506.43]),),
 (None,))

In [34]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.872, 557428.288, 8781219.303000001, 8842069.696999999)

In [21]:
chain_CTCOR.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function median at 0x7f0919b19d30>,
                          spacing=1000)),
             ('spline', Spline())])

In [22]:
chain_CTCOR.score(*test)

0.8139888924412879

In [23]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR, test_size=0.1, spacing=1000)

In [38]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=3)
plt.plot(test[0][0], test[0][1], '.r', markersize=3)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497270.872, 557428.288, 8781219.303000001, 8842069.696999999)

In [39]:
chain_CTCOR.fit(*train)

Chain(steps=[('trend', Trend(degree=1)),
             ('reduce',
              BlockReduce(reduction=<function median at 0x7f7764089ca0>,
                          spacing=1000)),
             ('spline', Spline())])

In [40]:
chain_CTCOR.score(*test)

0.7410376747452522

In [45]:
cv     = vd.BlockKFold(spacing=1000,
                  n_splits=50,
                  shuffle=True)

In [46]:
scores = vd.cross_val_score(chain_CTCOR,
                            coordinates,
                            g_rparaim.CTCOR,
                            cv=cv)

In [47]:
scores

array([0.74443574, 0.73913431, 0.59537797, 0.73120516, 0.66995736,
       0.74138783, 0.76763605, 0.71024327, 0.73332375, 0.77662805,
       0.70251326, 0.68903827, 0.68629422, 0.62909294, 0.71042494,
       0.74606402, 0.7101058 , 0.70248752, 0.75710194, 0.79936859,
       0.69332837, 0.77764073, 0.72224375, 0.76944521, 0.73784216,
       0.66416538, 0.77159434, 0.64732369, 0.70954701, 0.68938262,
       0.80596451, 0.74030454, 0.68423108, 0.71958288, 0.66572751,
       0.77499286, 0.78737367, 0.6471058 , 0.72912171, 0.75675404,
       0.71656058, 0.79560237, 0.7341918 , 0.72548906, 0.69692162,
       0.72413802, 0.6927257 , 0.69942104, 0.81977245, 0.68069013])

In [48]:
plt.figure()
plt.hist(scores, bins ='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  3.,  8., 13., 12.,  9.,  4.]),
 array([0.59537797, 0.62743433, 0.65949068, 0.69154704, 0.72360339,
        0.75565974, 0.7877161 , 0.81977245]),
 <BarContainer object of 7 artists>)

In [70]:
geof = CTCOR_grid.to_dataframe()
geof.reset_index(inplace=True)

In [71]:
geof

,easting,northing,CTCOR
0,500005.30,8.783985e+06,1659.578181
1,500005.30,8.784185e+06,1634.598105
2,500005.30,8.784385e+06,1599.014773
3,500005.30,8.784584e+06,1551.296070
4,500005.30,8.784784e+06,1492.718534
...,...,...,...
76167,554693.86,8.838505e+06,1171.607396
76168,554693.86,8.838705e+06,1173.625707
76169,554693.86,8.838904e+06,1174.655027
76170,554693.86,8.839104e+06,1174.081106


In [72]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 200,pixel_register=True)